In [1]:
import awswrangler as wr
from transformers import AutoTokenizer
from transformers import DataCollatorWithPadding
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer
import numpy as np


import evaluate

In [2]:
df = wr.s3.read_parquet(path="s3://amazon-reviews-eafit/sample/")

In [3]:
df.head()

,marketplace,customer_id,review_id,product_id,product_parent,product_title,product_category,star_rating,helpful_votes,total_votes,vine,verified_purchase,review_headline,review_body,review_date,category,sentiment
0,US,16849905,RBLJ34JHSBQ74,B0145P8DL8,391342641,AW Fashion's Under New Management - Funny Newl...,Apparel,5.0,0.0,0.0,N,N,Nice shirt,Great quality shirt. Not thin or see through. ...,2015-08-31,apparel,positive
1,US,16260349,R3RG81P69KZ6GF,B013G961KY,798311880,Coffee And Books - Ladies T Shirt Deep_royal L,Apparel,1.0,0.0,0.0,N,Y,One Star,Fit until I washed it for the first time. It ...,2015-08-31,apparel,negative
2,US,48175141,R2NT75EQ2D1LKI,B013FNK6PC,80644824,Ezcosplay Women Off Shouder T-shirt Hollow Out...,Apparel,3.0,9.0,10.0,N,N,Runs small,"This fit is very off on this shirt, I normally...",2015-08-30,apparel,neutral
3,US,27445392,R2S400P2951R85,B013FFIKBM,447463454,Tulle Shirts and Blouses for Women,Apparel,5.0,0.0,0.0,N,N,I really like this top,I really like this top! The high-low hem make...,2015-08-24,apparel,positive
4,US,51748675,RWJEFCBVMEO1C,B013CZXJRK,259261920,Women's Fashion Below the Knee Offic Pencil Sk...,Apparel,5.0,0.0,0.0,N,N,LOVE this stylish pencil skirt!,[[VIDEOID:a417b5ce461cd9d7b72d062da2a7c928]] T...,2015-08-25,apparel,positive


In [ ]:
tokenizer = AutoTokenizer.from_pretrained("distilbert/distilbert-base-uncased")

In [ ]:
def preprocess_function(examples):
    return tokenizer(examples["sentiment"], truncation=True)

In [ ]:
tokenized_df = df.map(preprocess_function, batched=True)

In [ ]:
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [ ]:
accuracy = evaluate.load("accuracy")

In [ ]:
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return accuracy.compute(predictions=predictions, references=labels)

In [ ]:
id2label = {0: "negative", 1: "neutral", 2: "positive"}
label2id = {"negative": 0, "neutral": 1, "positive": 2}